# Basic exercise 9

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft2, fftshift, ifft2, ifftshift

This is a basic exercise in the course Python For Scientists. 
The aim is to get you acquainted with the syntax of `scipy` and `numpy` and give you the necessary skills to tackle more serious problems later on.

Of course these problems can be solved very easily by using AI tools. However, since the goal is to teach you the basics, it is not recommended to use AI. Try to solve them independetly instead.

## FFT in solid state physics
In solid state physics, diffraction of X-rays or electrons is used to study the atomic structure of ordered (crystalline) materials. 
When X-rays or electrons pass through the lattice of such a material, they are diffracted and produce a diffraction pattern on a detector.
This diffraction pattern can be seen as a direct measurement of the Fourier transform of the atomic structure of the material. 
This information can (after again Fourier transforming it) be used to obtain an atomic model of the material that is being investigated.
<p align="center">
    <img src="Xray pattern.png" alt="drawing" width="500" align="center"/>
</p>

In this exercise we will investigate some properties of the Discrete Fourier Transform (DFT) in two dimensions on such patterns, and based on the insight that we get from this, we will try to learn something about the difference between two materials.

(Image source: Möttönen, N. (2021). *Xray pattern* [png image]. Aalto University Wiki. URL: https://wiki.aalto.fi/pages/viewpage.action?pageId=119605795 )

### Part 1
Numpy provides a way to efficiently store arrays in an .npy-file. 
In `pattern.npy`, a numpy array is stored that contains a pattern of squares. 
Load this array and visualize it with `imshow`.

In [ ]:
# Implement your solution here

In [ ]:
pattern = np.load("pattern.npy")
plt.close()
plt.imshow(pattern)
plt.show()

### Part 2
Calculate the DFT of this pattern and visualise the frequency space. 
You can create a better view on the frequency space by shifting it with `fftshift`, so that the zero-frequency is in the middle of the image.

Can you go back from frequency space to real space?

In [ ]:
# Implement your solution here

In [ ]:
freqspace = fft2(pattern)
plt.close()
plt.imshow(np.log(np.abs(fftshift(freqspace))))
plt.show()

In [ ]:
inverse = ifft2(freqspace)
plt.close()
plt.imshow(np.real(inverse))
plt.show()

## Part 3
You notice that there is noise present in the pattern. 
This can be filtered by removing some high frequencies from the DFT, for example by multplying with a box window in frequency space.
This box window is a numpy array of the same size as the frequency space, with values 1 in a square box around the zero frequency and values 0 everywhere else. 
Write a function that creates such a box window.
This function should have a parameter that defines the width of the window.

Create a box window of width 10 and use it to filter noise from the pattern. 
Remember that you might have to shift the frequency space in order to have the zero-frequency in the middle of the image! 
You can shift it back to the original configuration with `ifftshift`.

What is the effect of the size of the box on the filtered pattern?
Explain your answer by calculating the DFT of the box window itself. 
What does the a multiplication with the box in frequency space mean in real space?

In [ ]:
# Implement your solution here

In [ ]:
print(np.shape(pattern))


def box_window(width):
    x = int((100 - width) / 2)
    y = x + width
    box = np.ones(np.shape(pattern))
    box[0:x, :] = 0
    box[:, 0:x] = 0
    box[y:, :] = 0
    box[:, y:] = 0
    return box

In [ ]:
width = 12
box = box_window(width)
plt.close()
plt.imshow(box)
plt.show()

In [ ]:
filtered_freqspace = ifftshift(fftshift(freqspace) * box)
plt.close()
plt.imshow(abs(fftshift(filtered_freqspace)))
plt.show()

In [ ]:
filtered_inv = ifft2(filtered_freqspace)
plt.close()
plt.imshow(abs(filtered_inv))
plt.show()

In [ ]:
# The smaller the box filter, the more smeared out the pattern becomes.
# A smaller box filters out more frequencies so the noise is removed better,
# but the pattern also becomes more blurry.

# This is because the fourier transform of the box filter is a 2D form
# of a sinc function (sin(x)/x), as you can see below.
# Multiplication with the box in frequency space means convolution with this
# 2D sinc function in real space.

In [ ]:
box_DFT = ifft2(box)
plt.imshow(np.abs(fftshift(box_DFT)))
plt.show()

### Part 4

In a paper that you can find [here](ElectronScatteringPaper.pdf), the authors looked at the diffraction patterns of two different materials: Ti$_{23}$Fe$_{26}$Al$_{51}$ and Ti$_{30}$Co$_{25}$Al$_{45}$. The diffraction patterns are shown here below in figures a and c respectively. Based on the knowledge you built in this exercise, can you infer which pattern was measured from the sample with the largest crystal size? Explain your answer.

<p align="center">
    <img src="pattern1.png" alt="drawing" width="300" align="center"/>
</p>
<p align="center">
    <img src="pattern2.png" alt="drawing" width="300" align="center"/>
</p>

(images source: Grytsiv, A. et al. (2003). *Crystal chemistry of the G-phases in the systems Ti–{Fe, Co, Ni}–Al with a novel filled variant of the Th6Mn23-type*. Intermetallics, 11(4), 351-359.)


*Write your answer here*

If you cut out a part of frequency space, the points in real space get blurred.
This works in two directions: if you cut out a part of real space, the points in frequency space get blurred.

In the case of a diffraction experiment, the diffraction pattern represents the frequency space and the atoms in the sample represent the real space.
In ideal (theoretical) circumstances, you would perform the experiment on an infinite piece of material where the periodic lattice of atoms is spread out in all of space. 
This would result in a diffraction pattern of perfect points.
When you use a physical crystal that is limited in size, the points in frequency space, so the diffraction pattern, gets blurred and smeared out. The smaller the crystal you use, the more pronounced this blurring will be.

This means that the pattern in figure c was created with the smalles crystal sample, since the points in the diffraction pattern are more smeared out. This can be verified in Table 1 in the paper.